# VISTA3D - 16.4 LTS  ML - CUDA12

https://catalog.ngc.nvidia.com/orgs/nvidia/teams/monaitoolkit/models/monai_vista3d

---

### Vista3D - Code License
This project includes code licensed under the Apache License 2.0. [LINK](https://github.com/Project-MONAI/VISTA/blob/main/vista3d/LICENSE)

### Vista3D - Model Weights License
The model weights are made available by NVIDIA under the NCLS v1 License. Please review the license terms to ensure compliance prior to download the model weights. [NVIDIA OneWay Noncommercial License](https://github.com/Project-MONAI/VISTA/blob/main/vista3d/NVIDIA%20OneWay%20Noncommercial%20License.txt)

---


In [0]:
%pip install -r vista3d/requirements.txt
%pip install ./artifacts/monailabel-htj2k-py3-none-any.whl --no-deps --force-reinstall
%pip install monai==1.5.1 pytorch-ignite --no-deps
%pip install simpleitk
#%pip install pylibjpeg>=2.0 pylibjpeg-openjpeg>=2.0

In [0]:
%pip install mlflow>=3.0 --upgrade

In [0]:
%pip install numpy<2 pylibjpeg>=2.0 pylibjpeg-openjpeg>=2.0

In [0]:
dbutils.library.restartPython()

In [0]:
%run ../config/proxy_prep

In [0]:
sql_warehouse_id, table, volume = init_widgets(show_volume=True)
model_uc_name, serving_endpoint_name = init_model_serving_widgets()

dbutils.widgets.text("use_service_principal", "False", label="5.0 Use Service Principal")
use_service_principal = dbutils.widgets.get("use_service_principal").lower() == "true"

volume_path = volume.replace(".","/")

In [0]:
init_env()

os.environ["DEST_DIR"] = f"/Volumes/{volume_path}/monai_serving/vista3d/"

In [0]:
from monailabel.datastore import utils
from monailabel.interfaces.app import MONAILabelApp

app = MONAILabelApp(
  app_dir="/Volumes/ema_rina/pixels_solacc_3_10/pixels_volume/tmp/monailabel_app/",
  studies="https://e2-demo-field-eng.cloud.databricks.com/", conf={})

datastore = app.datastore()
datastore._convert_to_nifti = False
indexed_series = app.datastore().list_images()
series_dir = app.datastore().get_image_uri(indexed_series[0])
print(series_dir)

In [0]:
%sh
ls /root/.cache/monailabel/dicom/4984da479a5068a7bc1325afce441f62/1.1.538.1.2.5926223.7.470.3168968872654886780

In [0]:
%sh
rm -rf /root/.cache/monailabel/dicom/

In [0]:
from monailabel.datastore.utils import convert_htj2k

output_dir = "/Volumes/ema_rina/pixels_solacc_3_10/pixels_volume/htj2k_encoded/"

convert_htj2k.convert_single_frame_dicom_series_to_multiframe(input_dir=series_dir, output_dir=output_dir ,convert_to_htj2k=True)

In [0]:
import pydicom
from pydicom.pixels.utils import pixel_array

file_path = "/Volumes/ema_rina/pixels_solacc_3_10/pixels_volume/1.2.156.14702.1.1000.16.0.20200311113603875/1.2.156.14702.1.1000.16.1.2020031111365289000020001.dcm"

ds = pydicom.dcmread(file_path, stop_before_pixels=True)

frame = pixel_array(file_path, index=10)



In [0]:
from vista3d.code.dbvista3dmodel import DBVISTA3DModel

model = DBVISTA3DModel(volumes_compatible=True)

In [0]:
from mlflow.models import infer_signature
from typing import Optional

input_examples = [
      { "input": { "action": "info" }},                   #retrieve informations about the monailabel server
      { "input": { "action": "activelearning/random" }},  #randomly return the next series_uid useful to label
      { "input": {                                        #train the model based on labelled series
        "train": {
          'name': 'train_01',
          'pretrained': True,
          'device': ['NVIDIA A10G'],
          'max_epochs': 50,
          'early_stop_patience': -1,
          'val_split': 0.2,
          'train_batch_size': 1,
          'val_batch_size': 1,
          'multi_gpu': True,
          'gpus': 'all',
          'dataset': 'SmartCacheDataset',
          'dataloader': 'ThreadDataLoader',
          'tracking': 'mlflow',
          'tracking_uri': '',
          'tracking_experiment_name': '',
          'model': 'segmentation'
          }
       }
      },
      { 'input': {                                        #train the model based on labelled series with mandatory fields
        'train': {
          'name': 'train_01',
          'pretrained': True,
          'max_epochs': 50,
          'val_split': 0.2,
          'train_batch_size': 1,
          'val_batch_size': 1,
          'gpus': 'all',
          'model': 'segmentation'
          }
       }
      },                      
      { 'input': {                                        #trigger the inference on a single DICOM series given the series uid, used in OHIF Viewer
        'infer': {
          'largest_cc': False,
          'device': ['NVIDIA A10G'],
          'result_extension': '.nrrd',
          'result_dtype': 'uint16',
          'result_compress': False,
          'restore_label_idx': False,
          'model': 'vista3d',
          'image': '1.2.156.14702.1.1000.16.1.2020031111365289000020001',
          'export_metrics': False,
          'export_overlays': False,
          'points': [[10,10,10],[20,20,20]], #list of x,y,z points
          'point_labels': [0,1],
          'pixels_table': "main.pixels_solacc.object_catalog"
          }
       }
      },
      { 'input': {                                        #trigger the inference on a single DICOM series given the series uid, used in OHIF Viewer with mandatory fields
        'infer': {
          'model': 'vista3d',
          'image': '1.2.156.14702.1.1000.16.1.2020031111365289000020001',
          'label_prompt': [1,26]
          }
       }
      },
      { 'input': {                                        #Return the file from the inference, used in OHIF Viewer
        'get_file': '/tmp/vista/bundles/vista3d/models/prediction/1.2.156.14702.1.1000.16.1.2020031111365289000020001/1.2.156.14702.1.1000.16.1.2020031111365289000020001_seg.nii.gz',
        'result_dtype': 'uint8'
       }
      },
      { 'series_uid': '1.2.156.14702.1.1000.16.1.2020031111365293700020003',
        'params' : {
          'label_prompt' : [1,26],
          'export_metrics': False,
          'export_overlays': False,
          'points': [[100,100,100],[200,200,200]],
          'point_labels': [0,1],
          'dest_dir': '/Volumes/main/pixels_solacc/pixels_volume/monai_serving/vista3d',
          'pixels_table': "main.pixels_solacc.object_catalog",
          'torch_device': 0
        }
      },
      { 'series_uid': '1.2.156.14702.1.1000.16.1.2020031111365293700020003',
       'params' : {},
      },
      { 'series_uid': '1.2.156.14702.1.1000.16.1.2020031111365293700020003'}
]

signature = infer_signature(input_examples, model_output="")
signature.inputs.to_json()

In [0]:
from common.utils import download_dcmqi_tools

# Download the dcmqi tool binary used for the conversion of nifti files to DICOM SEG files
download_dcmqi_tools("./artifacts")

In [0]:
# === OPTIONAL | Requires GPU Enabled cluster ===

try:
  import torchvision
  import pandas as pd
  import json

  label_prompt = ["liver", "hepatic tumor"]

  label_dict_path = "vista3d/code/vista3d_bundle/data/jsons/label_dict.json"
  label_dict = json.load(open(label_dict_path))
  label_index = [label_dict[label.strip()] for label in label_prompt if label.strip() in label_dict]

  # Pick one of the series_uid available in the pixels' catalog table
  series_uid = "1.2.156.14702.1.1000.16.1.2020031111365295300020004"

  input = { "series_uid": series_uid, "params": {
    "label_prompt": [1,26],
    "export_metrics": True,
    "export_overlays": False,
    "dest_dir": f"/Volumes/{volume_path}/monai_serving/vista3d",
    "pixels_table" : table
    }
  }

  df = pd.DataFrame([input_examples[5]])

  # This step will download the VISTA3D Model bundle scripts and model weights to the local disk
  # This step will automatically download in the ./bin folder the itkimage2segimage binary required for the conversion of nifti files to DICOM SEG files

  model.load_context(context=None)
  result = model.predict(None, df)
except ImportError as e:
  print(e,", skipping model test")

In [0]:
%sh
rm -rf /root/.cache/monailabel/

In [0]:
%sh
ls -alh /root/.cache/monailabel/dicom/6be462ca0dea7f5aa796d3c781ccfd6b/1.1.538.1.2.5926223.7.470.3168968872654886780


In [0]:
import sys

sys.path.append("./vista3d/code/")

In [0]:
from monai.transforms import LoadImageD
from monailabel.transform.reader import NvDicomReader

# Initialize the loader
loader = LoadImageD(keys=['image'], image_only=True)

# Register a specific reader instance
loader.register(NvDicomReader()) 

In [0]:
from vista3d_bundle.scripts.infer import InferClass
from common.utils import series_to_nifti, init_dicomweb_datastore
import os

datastore = init_dicomweb_datastore(os.environ["DATABRICKS_HOST"], os.environ['DATABRICKS_TOKEN'], os.environ["DATABRICKS_WAREHOUSE_ID"], table)

series_uid = datastore.list_images()[1]

datastore._convert_to_nifti = False
series_dir = datastore.get_image_uri(series_uid)
print(series_dir)

output_dir = f"/tmp/vista3d/{series_uid}/"

vista3d_model = InferClass("./vista3d/code/vista3d_bundle/configs/infer.yaml", torch_device="cuda", file_ext=".dcm", output_dir=output_dir)
vista3d_model.infer("/root/.cache/monailabel/dicom/6be462ca0dea7f5aa796d3c781ccfd6b/3.4.995.1.0.9199491.4.131.1796546694675378669/", label_prompt=None, point=None, point_label=None, save_mask=True)

print(f"Inference completed on image: {series_dir}")

In [0]:
%sh
rm -rf /root/.cache/monailabel/dicom/

In [0]:
%sh
ls -alh /tmp/vista3d/1.2.156.14702.1.1000.16.1.2020031111365295300020004/1.2.156.14702.1.1000.16.1.2020031111365295300020004/

In [0]:
%sh
file /root/.cache/monailabel/dicom/6be462ca0dea7f5aa796d3c781ccfd6b/3.4.995.1.0.9199491.4.131.1796546694675378669/3.4.995.1.0.9199491.4.131.3345539409509941649.dcm


In [0]:
import pydicom
from pydicom.pixels.utils import pixel_array

path = "/root/.cache/monailabel/dicom/6be462ca0dea7f5aa796d3c781ccfd6b/3.4.995.1.0.9199491.4.131.1796546694675378669/3.4.995.1.0.9199491.4.131.3345539409509941649.dcm"

#import nibabel as nib

#nifti_img = nib.load(path)
#frame = nifti_img.get_fdata()[..., 100]

ds = pydicom.dcmread(path)

frame = pixel_array(path, index=0)

import matplotlib.pyplot as plt

plt.imshow(frame)
plt.axis('off')
plt.show()

In [0]:
import mlflow

# Save the function as a model
with mlflow.start_run():
    mlflow.pyfunc.log_model (
        "DBVISTA3DModel",
        python_model=DBVISTA3DModel(),
        conda_env="./vista3d/conda.yaml",
        signature=signature,
        code_paths=["./vista3d", "./common", "./lib"],
        artifacts={
            "monailabel-0.8.5": "./artifacts/monailabel-0.8.5-py3-none-any.whl",
            "itkimage2segimage": "./artifacts/itkimage2segimage"
        }
    )
    run_id = mlflow.active_run().info.run_id
    print(run_id)

In [0]:
model_uri = "runs:/{}/DBVISTA3DModel".format(run_id)
latest_model = mlflow.register_model(model_uri, model_uc_name)

In [0]:
# Define scope and key names for the credentials

scope_name = "pixels_scope"
sp_name = "pixels_sp"

sp_id_key = "pixels_sp_id"
sp_app_id_key = "pixels_sp_app_id"
sp_secret_key = "pixels_sp_secret"
token_key = "pixels_token"

m2m_client = None

delete_all_sercrets = False

# Create Service Principal and generate access token
## Fallback on Personal Access Token

In [0]:
from dbx.pixels.m2m import DatabricksM2MAuth
from databricks.sdk import WorkspaceClient

if use_service_principal:
    try:
        m2m_client = DatabricksM2MAuth(
            principal_name=sp_name,
            account_api_token=os.environ["DATABRICKS_TOKEN"],

            secrets_scope_name=scope_name,
            secrets_client_id_key=sp_id_key,
            secrets_client_app_id_key=sp_app_id_key,
            secrets_client_secret_key=sp_secret_key,

            workspace_url=os.environ["DATABRICKS_HOST"]
        )
        m2m_client.grant_permissions(table, volume)
    except Exception as e:
        print(e)
else: 
    # Create Personal Access Token | Not needed if service principal is used ===
    w = WorkspaceClient()

    if scope_name not in [scope.name for scope in w.secrets.list_scopes()]:
        w.secrets.create_scope(scope=scope_name)

    token = w.tokens.create(comment=f'pixels_serving_endpoint_token')

    w.secrets.put_secret(scope=scope_name, key=token_key, string_value=token.token_value)

    print(f"PAT created and saved in {token_key} secret")


In [0]:
# Sample script to cleanup secrets | Max 5 secrets are allowed per Service Principal | Disable by default

if delete_all_sercrets:
    from dbx.pixels.m2m import DatabricksM2MAuth

    list_secrets = DatabricksM2MAuth.list_service_principal_secrets(
        workspace_url=os.environ["DATABRICKS_HOST"], 
        account_api_token=os.environ["DATABRICKS_TOKEN"], 
        client_id=dbutils.secrets.get(scope_name, sp_id_key)
    )['secrets']

    for secret in list_secrets:
        DatabricksM2MAuth.delete_service_principal_secret(
            workspace_url=os.environ["DATABRICKS_HOST"], 
            account_api_token=os.environ["DATABRICKS_TOKEN"], 
            client_id=dbutils.secrets.get(scope_name, sp_id_key), secret_id=secret['id'])

In [0]:
from mlflow.deployments import get_deploy_client

client = get_deploy_client("databricks")

model_version = latest_model.version

secret_template = "secrets/{scope}/{key}"

token = "{{" + secret_template.format(scope=scope_name, key=token_key) + "}}"
client_app_id = "{{" + secret_template.format(scope=scope_name, key=sp_app_id_key) + "}}"
client_secret = "{{" + secret_template.format(scope=scope_name, key=sp_secret_key) + "}}"

conf_vars = {
    'DATABRICKS_HOST': os.environ["DATABRICKS_HOST"],
    'DATABRICKS_PIXELS_TABLE': os.environ["DATABRICKS_PIXELS_TABLE"],
    'DATABRICKS_WAREHOUSE_ID': os.environ["DATABRICKS_WAREHOUSE_ID"],
    'DEST_DIR': os.environ["DEST_DIR"]
}

if not m2m_client:
    conf_vars['DATABRICKS_TOKEN'] = token
else:
    conf_vars['DATABRICKS_SCOPE'] = scope_name
    conf_vars['CLIENT_APP_ID'] = client_app_id
    conf_vars['CLIENT_SECRET'] = client_secret

endpoint = client.create_endpoint(
    name=serving_endpoint_name,
    config={
        "served_entities": [
            {
                'entity_name': model_uc_name,
                "entity_version": model_version,
                "workload_size": "Small",
                "workload_type": "GPU_MEDIUM",
                "scale_to_zero_enabled": True,
                'environment_vars': conf_vars,
            }
        ]
    }
)

print("SERVING ENDPOINT CREATED:", serving_endpoint_name)


## Test the connection and execute inference using Serving Endpoint with Vista3D model

NOTE: Serving Endpoint creation will take ~ 30 minutes to complete

In [0]:
import time
from mlflow.deployments import get_deploy_client

client = get_deploy_client("databricks")

def wait_for_endpoint_ready(endpoint_name, client, timeout=2100, interval=10):
    start_time = time.time()
    while time.time() - start_time < timeout:
        endpoint_status = client.get_endpoint(endpoint_name)
        if endpoint_status['state']['ready'] == "READY":
            print(f"Endpoint {endpoint_name} is ready.")
            return
        time.sleep(interval)
    raise TimeoutError(f"Endpoint {endpoint_name} did not become ready within {timeout} seconds.")

wait_for_endpoint_ready(serving_endpoint_name, client)

In [0]:
from dbx.pixels.modelserving.vista3d.servingendpoint import Vista3DMONAITransformer

df = spark.table(table)

df_monai = Vista3DMONAITransformer(table=table, destDir=os.environ["DEST_DIR"], endpoint_name=serving_endpoint_name, exportMetrics=True).transform(df)

display(df_monai.filter('series_uid = "1.2.156.14702.1.1000.16.1.2020031111365289000020001"'))

# Test performance using noop
#df_monai.repartition(4).write.format("noop").mode("overwrite").save()

### Initialize a Vista3DGPUTransformer to process the pixels' catalog table using GPU resources.

In [0]:
import torch
from dbx.pixels.modelserving.vista3d.gpu import Vista3DGPUTransformer

gpuCount = int(spark.conf.get("spark.executor.resource.gpu.amount","0") or torch.cuda.device_count())
nWorkers = (int(spark.conf.get("spark.databricks.clusterUsageTags.clusterWorkers")) or 1)
tasksPerGpu = int(spark.conf.get("spark.task.resource.gpu.amount","1"))

df = spark.table(table)

df_monai = Vista3DGPUTransformer(inputCol="meta", 
                                 table=table, 
                                 destDir=os.environ["DEST_DIR"], 
                                 sqlWarehouseId=os.environ["DATABRICKS_WAREHOUSE_ID"], 
                                 labelPrompt=None, exportMetrics=True, exportOverlays=False, 
                                 secret=os.environ["DATABRICKS_TOKEN"], 
                                 host=os.environ["DATABRICKS_HOST"], 
                                 gpuCount=gpuCount, nWorkers=nWorkers, tasksPerGpu=tasksPerGpu).transform(df)

display(df_monai)

# Test performance using noop
#df_monai.write.format("noop").mode("overwrite").save()

In [0]:
%sql
-- Requires Databricks Runtime 15.2 and above or Serverless
-- Sample query to illustrate how to use the ai_query function to query vista3d model in serving endpoint 
with ct as (
  select distinct(meta:['0020000E'].Value[0]) as series_uid
  from ${table}
  where meta:['00080008'] like '%AXIAL%'
)

select series_uid, parse_json(ai_query(
  endpoint => '${serving_endpoint_name}',
  request => named_struct(
      'series_uid', series_uid,
      'params', named_struct(
                    'export_metrics', True,
                    'export_overlays', True
                )
  ),
  returnType => 'STRING'
)) as result from ct